<a href="https://colab.research.google.com/github/psyduck1203/ML-ProjectKart/blob/add-license-1/Email_Classification/Model/email_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
mails = pd.read_csv("/content/SMS_train.csv",encoding='ISO-8859-1')

In [3]:
mails.head()

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam


In [4]:
mails.tail()

,S. No.,Message_body,Label
952,953,hows my favourite person today? r u workin har...,Non-Spam
953,954,How much you got for cleaning,Non-Spam
954,955,Sorry da. I gone mad so many pending works wha...,Non-Spam
955,956,Wat time ü finish?,Non-Spam
956,957,Just glad to be talking to you.,Non-Spam


In [5]:
#get rows and columns size
mails.shape

(957, 3)

In [6]:
# get null values
mails.isnull().sum()

S. No.          0
Message_body    0
Label           0
dtype: int64

In [7]:
#check if there are duplicates
mails.duplicated().sum()

0

In [8]:
#lets check the way the first 4 rows looks
for each in mails['Message_body'].head(4):
    print("\n\n")
    print(each)




Rofl. Its true to its name



The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free



Pity, * was in mood for that. So...any other suggestions?



Will ü b going to esplanade fr home?


In [9]:
#check if we have urls
pattern = r"(?P<protocol>https?)://(?P<domain>[\w\.\-]+)/?(?P<path>.*)"
urls = mails['Message_body'].str.extract(pattern)

In [10]:
urls.sum()
#thre are no urls

Series([], dtype: float64)

In [11]:
#removing emails keywords
#we will replace with ""
import re

pattern = r"e[-\s]?mail"
email_uniform = mails['Message_body'].str.replace(pattern , "email" , flags=re.I)

In [12]:
#working with numbers
pattern =r"([0-9]+)"
nums = mails["Message_body"].str.extractall(pattern)

In [13]:
import string
import re
x = string.punctuation
pattern =r"([0-9]+)"
def clearAll(Message_body):
    Message_body = Message_body.replace(string.punctuation , "")
    
    return Message_body

In [14]:
mails.head(4).apply(clearAll)

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam


In [15]:
# remove any non chars
for c in range(mails.shape[0]):
    mails['Message_body'].iloc[c] = [x for x in mails['Message_body'].iloc[c] if x not in string.punctuation]
    mails['Message_body'].iloc[c] =''.join(mails['Message_body'].iloc[c])

In [16]:
# pattern = r"([0-9]+)"
#remove the digits
number = ["1" , "2","3" ,'4' ,'5','6','7','8','9' ,"0"]
for c in  range(mails.shape[0]):
    mails['Message_body'].iloc[c] = [x for x in mails['Message_body'].iloc[c] if x not in number]
    mails['Message_body'].iloc[c] =''.join(mails['Message_body'].iloc[c])

In [17]:
#do vectorization and covert it to matrix
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

text = vec.fit_transform(mails['Message_body'])


In [18]:
#get the labels
y = mails['Label']
text.shape


(957, 3022)

In [19]:
#do splitliting the data 80 to 20
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text , y , test_size = 0.2 , random_state = 0)

In [20]:
#training using naibayes classifier(multinomial)
from sklearn.naive_bayes import MultinomialNB
clf1 = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
#cross validation for naive bayes algo
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf1 , X = text, y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[1.         0.95833333 0.96875    1.         0.94791667 0.92708333
 0.94791667 0.98947368 0.96842105 0.93684211]
Mean for cross val is 0.9644736842105264
std for the cross val is 0.02435287022912144


In [22]:
#do training baesd on multinomialnb
clf1.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred = clf1.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.99      1.00      1.00       673
        Spam       0.99      0.96      0.97        92

    accuracy                           0.99       765
   macro avg       0.99      0.98      0.98       765
weighted avg       0.99      0.99      0.99       765




Confudion Matrix  
 [[672   1]
 [  4  88]]



Accuracy score   0.9934640522875817


In [24]:
#checking score based on set test
pred = clf1.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.98      0.99      0.98       162
        Spam       0.93      0.87      0.90        30

    accuracy                           0.97       192
   macro avg       0.95      0.93      0.94       192
weighted avg       0.97      0.97      0.97       192




Confudion Matrix  
 [[160   2]
 [  4  26]]



Accuracy score   0.96875


In [25]:
#randomclassifier model
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier()


In [26]:
#cross validation for random classifeir
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf2 , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.95833333 0.92708333 0.96875    0.96875    0.92708333 0.92708333
 0.92708333 0.93684211 0.93684211 0.93684211]
Mean for cross val is 0.9414692982456142
std for the cross val is 0.016324600290308793


In [27]:
clf2.fit(x_train ,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred2 = clf2.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred2)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred2)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred2)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       1.00      1.00      1.00       673
        Spam       1.00      1.00      1.00        92

    accuracy                           1.00       765
   macro avg       1.00      1.00      1.00       765
weighted avg       1.00      1.00      1.00       765




Confudion Matrix  
 [[673   0]
 [  0  92]]



Accuracy score   1.0


In [29]:
#checking score based on set test
pred1 = clf2.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred1)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred1)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred1)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.91      1.00      0.95       162
        Spam       1.00      0.47      0.64        30

    accuracy                           0.92       192
   macro avg       0.96      0.73      0.79       192
weighted avg       0.92      0.92      0.90       192




Confudion Matrix  
 [[162   0]
 [ 16  14]]



Accuracy score   0.9166666666666666


In [30]:
#lets check for boosting algo
from xgboost import XGBClassifier
clf3 = XGBClassifier()

In [31]:
#cross validation
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf3 , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.96875    0.95833333 0.97916667 1.         0.94791667 0.9375
 0.89583333 0.96842105 0.92631579 0.95789474]
Mean for cross val is 0.9540131578947367
std for the cross val is 0.027690917210666563


In [32]:
clf3.fit(x_train , y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [33]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred3 = clf3.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred3)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred3)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred3)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.98      1.00      0.99       673
        Spam       0.99      0.84      0.91        92

    accuracy                           0.98       765
   macro avg       0.98      0.92      0.95       765
weighted avg       0.98      0.98      0.98       765




Confudion Matrix  
 [[672   1]
 [ 15  77]]



Accuracy score   0.9790849673202614


In [34]:
#checking score based on set test
pred3 = clf3.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred3)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred3)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred3)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.96      0.99      0.98       162
        Spam       0.96      0.77      0.85        30

    accuracy                           0.96       192
   macro avg       0.96      0.88      0.91       192
weighted avg       0.96      0.96      0.96       192




Confudion Matrix  
 [[161   1]
 [  7  23]]



Accuracy score   0.9583333333333334
